##### This notebook imports source data for this project into BigQuery. 
##### The source data is first exported in GCS in CSV format. The BigQuery source table is "bigquery-public-data.ml_datasets.ulb_fraud_detection". This is a table of credit card transactions that are classified as fradulant, Class =1 or normal, Class = 0.

##### This table has 284807 credit card transaction classified as fradulent or normal in the column Class. In order to protect confidentiality the original features have been transformed using PCA(Principal component analysis) into 28 features named V1,V2,....,V28(float). 
##### Two descriptive features are provided without transformation by PCA.
##### Time (integer): is the seconds elapsed between the transaction and the eraliest transaction in the table
##### Amount (float): is the value of the transaction.

#### Preparation of Data
##### transaction_id (string): a unique id for row/transaction
##### splits (string): this divides the transactions into the set of TRAIN(80%), VALIDATE(10%), TEST(10%)

In [4]:
# Setup
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'ml-pipeline-project-401216'

In [9]:
REGION = "us-central1"
EXPERIMENT = "01"
SERIES = "01"

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = "fraud"
BQ_TABLE = "fraud"

# sourcing data from bigquery-public-data.ml_datasets.ulb_fraud_detection
BQ_SOURCE = "bigquery-public-data.ml_datasets.ulb_fraud_detection"

In [10]:
# packages and clients:
from google.cloud import storage
from google.cloud import bigquery

gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

In [11]:
BUCKET = PROJECT_ID + '-bucket1'
BUCKET

'ml-pipeline-project-401216-bucket1'

In [12]:
# Source data in GCS Storage bucket

file = "{}/{}/data/{}.csv".format(SERIES, EXPERIMENT, BQ_TABLE)
file

'01/01/data/fraud.csv'

In [15]:
# Exporting the dataset into GCS bucket

bucketDef = gcs.bucket(BUCKET)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print("The file is already created at: gs://{}/{}".format(bucketDef.name, file))
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = ["gs://{}/{}".format(bucketDef.name, file)])
    print("Creating the export....")
    extract.result()
    print("Exported the table to gs://{}/{}".format(bucketDef.name, file))
    

The file is already created at: gs://ml-pipeline-project-401216-bucket1/01/01/data/fraud.csv


In [16]:
# list files in the bucket
list(bucketDef.list_blobs(prefix = "{}/{}".format(SERIES, EXPERIMENT)))

[<Blob: ml-pipeline-project-401216-bucket1, 01/01/data/fraud.csv, 1697184037197809>]

In [52]:
# Checking if datasets exists in bigQuery
datasets = list(bq.list_datasets())
for sets in datasets:
    print(sets.dataset_id)

fraud


In [54]:
# create dataset if missing

ds = bigquery.Dataset("{}.{}".format(BQ_PROJECT ,BQ_DATASET))
ds.location = REGION
ds.labels = {'experiment' : f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

In [76]:
# creating table in the dataset

from google.cloud.exceptions import NotFound

try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    
except NotFound as error:
    print("Creating table....")
    destination = bigquery.TableReference.from_string("{}.{}.{}".format(BQ_PROJECT, BQ_DATASET, BQ_TABLE))
    
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment' : f'{EXPERIMENT}'}
    )
        
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print("Table created successfully: {}.{}.{}".format(BQ_PROJECT, BQ_DATASET, BQ_TABLE))
    

Creating table....
Table created successfully: ml-pipeline-project-401216.fraud.fraud


In [79]:
# Viewing sample of table from bigquery

query = f"""
    SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
    #LIMIT 5
"""

bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,152770,2.147860,0.168196,-1.815561,0.074078,0.759384,-0.777666,0.500592,-0.421479,0.311255,...,-0.186421,-0.364685,0.090908,-1.053785,0.166506,-0.427903,-0.018293,-0.063763,0.00,0
1,153526,2.112608,0.134436,-2.485294,0.319255,0.901347,-1.135053,0.806315,-0.388994,-0.281632,...,0.302005,0.952619,-0.139831,0.838490,0.634229,0.670702,-0.138295,-0.098389,0.00,0
2,154027,1.866596,0.292704,0.324495,3.875621,-0.307901,0.313782,-0.534334,0.124539,-0.286059,...,0.177422,0.659361,0.197373,-0.031893,-0.243696,0.031600,0.022030,-0.032014,0.00,0
3,155796,2.118083,0.033673,-2.502337,0.351595,0.736112,-1.292992,0.756271,-0.364783,-0.055416,...,0.275385,0.863478,-0.198812,-0.354270,0.606328,0.736778,-0.138825,-0.112377,0.00,0
4,157509,2.016066,-0.481205,-0.413139,0.354444,-0.563848,-0.047718,-0.781380,0.107206,1.355272,...,0.140128,0.589605,0.207411,0.642611,-0.278033,0.548471,-0.023257,-0.044493,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142398,153653,-5.192496,3.164721,-5.047679,2.246597,-4.011781,-0.638908,-2.873463,1.576318,-2.861986,...,1.167244,-1.006617,0.774562,0.063397,-0.390658,1.884741,-1.742558,-0.082216,247.86,1
142399,41138,-4.595617,5.083690,-7.581015,7.546033,-6.949165,-1.729185,-8.190192,2.714670,-7.083169,...,2.248971,0.566844,0.033744,0.591783,0.334229,0.386801,2.163898,0.983104,340.11,1
142400,70270,-1.512516,1.133139,-1.601052,2.813401,-2.664503,-0.310371,-1.520895,0.852996,-1.496495,...,0.729828,0.485286,0.567005,0.323586,0.040871,0.825814,0.414482,0.267265,318.11,1
142401,25095,1.192396,1.338974,-0.678876,3.123672,0.643245,-1.184323,0.397586,-0.253499,0.411135,...,-0.377503,-0.889597,-0.074208,0.035446,0.550578,-0.027171,-0.024921,0.073605,3.12,1


In [83]:
# Reviwing data in Bigquery

query = f"""   
    SELECT CLASS FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""

df = bq.query(query = query).to_dataframe()
df['CLASS'].value_counts()

0    284315
1       492
Name: CLASS, dtype: Int64

In [84]:
df['CLASS'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: CLASS, dtype: Float64

In [85]:
# Preparing data for Analysis
# create a prepped version of data with test/train splits using SQL DDL

query = f"""
    CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
    WITH add_id AS (SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
    SELECT *,
        CASE
            WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN"
            WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
            ELSE "TEST"
        END AS splits
    FROM add_id
"""

job = bq.query(query = query)
job.result()

In [86]:
(job.ended - job.started).total_seconds()

8.468

In [87]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

70.632136 MB


In [88]:
query = f"""
    SELECT splits, count(*) AS Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
    GROUP BY splits
"""

bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,TRAIN,228399,80.194307
1,VALIDATE,28334,9.948491
2,TEST,28074,9.857202


In [89]:
query = f"""
    SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
    LIMIT 5
"""

data = bq.query(query = query).to_dataframe()

In [90]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,33836,1.028170,0.084019,1.507280,2.739710,-0.510364,1.202849,-0.738733,0.448088,0.084509,...,-0.120605,0.093995,0.539835,0.222884,0.066238,0.014932,0.0,0,a3b15a6a-e5d5-4e4b-b6d0-9aa5f0afdca9,TEST
1,67443,1.263510,0.102121,-0.005493,0.486423,-0.130838,-0.508774,0.090794,-0.066110,0.009589,...,-0.153878,0.058095,0.661726,0.559488,-0.059897,-0.014047,0.0,0,97485a50-de5c-4d5b-8f00-4c5ae4f8878f,TEST
2,132245,-2.124347,-0.088218,0.790238,3.207753,3.030001,1.358073,0.759941,0.472682,-2.295062,...,0.814931,-1.687168,1.040174,0.445021,0.097742,-0.140581,0.0,0,7f7e440c-bd0e-4c7e-bb5a-fffb690ae6d7,TEST
3,35881,1.113746,0.157606,1.508637,2.909750,-0.788296,0.397501,-0.581851,0.182217,0.401191,...,-0.133046,0.451944,0.630394,0.211634,0.051267,0.027795,0.0,0,dbc10e46-0738-49f5-a1b7-3bce4039de0f,TEST
4,160815,2.085267,-0.289692,-3.073191,-0.682208,2.707364,3.125480,-0.155035,0.657321,0.235257,...,0.121871,0.761006,0.215527,0.582324,-0.076993,-0.084356,0.0,0,14fe5439-4271-4924-af28-a90759cb9385,TEST
